In [1]:
from fastai.collab import *
from fastai.tabular.all import *

In [5]:
path = untar_data(URLs.ML_100k)

In [6]:
path.ls()

(#23) [Path('/home/anasys/.fastai/data/ml-100k/u4.test'),Path('/home/anasys/.fastai/data/ml-100k/u5.base'),Path('/home/anasys/.fastai/data/ml-100k/u3.base'),Path('/home/anasys/.fastai/data/ml-100k/u.info'),Path('/home/anasys/.fastai/data/ml-100k/u.data'),Path('/home/anasys/.fastai/data/ml-100k/ub.test'),Path('/home/anasys/.fastai/data/ml-100k/u.genre'),Path('/home/anasys/.fastai/data/ml-100k/allbut.pl'),Path('/home/anasys/.fastai/data/ml-100k/u1.base'),Path('/home/anasys/.fastai/data/ml-100k/u3.test')...]

In [9]:
ratings = pd.read_csv(path / 'u.data',
                      delimiter='\t',
                      header=None,
                      names=['user', 'movie', 'rating', 'timestamp'])

In [11]:
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [17]:
# science-fiction, action, old movies
last_skywalker = np.array([0.98, 0.9, -0.9])
casablanca = np.array([-0.99, -0.3, 0.8])

In [18]:
user1 = np.array([0.9, 0.8, -0.6])

In [19]:
(user1 * last_skywalker).sum()

2.1420000000000003

In [20]:
(user1 * casablanca).sum()

-1.611

In [23]:
movies = pd.read_csv(path / 'u.item',
                     delimiter='|',
                     encoding='latin-1',
                     usecols=(0, 1),
                     names=('movie', 'title'),
                     header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [25]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [27]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

,user,title,rating
0,877,Muriel's Wedding (1994),4
1,397,East of Eden (1955),4
2,592,187 (1997),3
3,429,"Wizard of Oz, The (1939)",3
4,40,"Full Monty, The (1997)",1
5,266,White Squall (1996),2
6,268,Raiders of the Lost Ark (1981),5
7,498,Chasing Amy (1997),2
8,17,"Rock, The (1996)",3
9,503,GoodFellas (1990),3


In [29]:
n_users = len(dls.classes['user'])
n_users

944

In [30]:
n_movies = len(dls.classes['title'])
n_movies

1665

In [31]:
n_factors = 5

In [32]:
user_factors = torch.randn(n_users, n_factors)
user_factors.shape

torch.Size([944, 5])

In [34]:
movie_factors = torch.randn(n_movies, n_factors)
movie_factors.shape

torch.Size([1665, 5])

In [44]:
one_hot_3 = one_hot(3, n_users).float()
user_factors.t() @ one_hot_3

tensor([ 0.4899, -0.2355, -0.0463,  0.9462,  0.5654])

In [46]:
user_factors.shape, one_hot_3.shape

(torch.Size([944, 5]), torch.Size([944]))

In [41]:
user_factors[3]

tensor([ 0.4899, -0.2355, -0.0463,  0.9462,  0.5654])

In [48]:
class Example:
    def __init__(self, a):
        self.a = a
    
    def say(self, x):
        return f'Hello {self.a}, {x}.'

In [49]:
ex = Example('Sylvan')
ex.say('nice to meet you')

'Hello Sylvan, nice to meet you.'

In [81]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors):
        self.user_factors = Embedding(n_users, n_factors)
        self.movie_factors = Embedding(n_movies, n_factors)
    
    def forward(self, x):
        # x[batch, 0] => user_id
        # x[batch, 1] => movie_id
        users = self.user_factors(x[:, 0])
        movies = self.movie_factors(x[:, 1])
        return (users * movies).sum(dim=1)

In [82]:
x, y = dls.one_batch()
x.shape, y.shape

(torch.Size([64, 2]), torch.Size([64, 1]))

In [83]:
x[:, 0].shape

torch.Size([64])

In [84]:
x[:, 1].shape

torch.Size([64])

In [85]:
user_factors = Embedding(n_users, n_factors)
users = user_factors(to_cpu(x[:, 0]))
users.shape

torch.Size([64, 5])

In [86]:
movie_factors = Embedding(n_movies, n_factors)
movies = movie_factors(to_cpu(x[:, 1]))
movies.shape

torch.Size([64, 5])

In [87]:
(users * movies).sum(dim=1).shape

torch.Size([64])

In [88]:
model = DotProduct(n_users, n_movies, 50)
model

DotProduct(
  (user_factors): Embedding(944, 50)
  (movie_factors): Embedding(1665, 50)
)

In [89]:
learn = Learner(dls, model, loss_func=MSELossFlat())

In [90]:
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,1.348482,1.292598,00:06
1,1.071086,1.102618,00:06
2,0.950296,0.965836,00:06
3,0.840710,0.874548,00:06
4,0.799524,0.858594,00:06


In [91]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0, 5.5)):
        self.user_factors = Embedding(n_users, n_factors)
        self.movie_factors = Embedding(n_movies, n_factors)
        self.y_range = y_range
    
    def forward(self, x):
        # x[batch, 0] => user_id
        # x[batch, 1] => movie_id
        users = self.user_factors(x[:, 0])
        movies = self.movie_factors(x[:, 1])
        return sigmoid_range((users * movies).sum(dim=1), *self.y_range)

In [92]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.980007,0.991965,00:06
1,0.851325,0.892387,00:06
2,0.671646,0.860641,00:06
3,0.483190,0.865426,00:06
4,0.376567,0.869997,00:06


In [93]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0, 5.5)):
        self.user_factors = Embedding(n_users, n_factors)
        self.user_bias = Embedding(n_users, 1)

        self.movie_factors = Embedding(n_movies, n_factors)
        self.movie_bias = Embedding(n_movies, 1)

        self.y_range = y_range
    
    def forward(self, x):
        # x[batch, 0] => user_id
        # x[batch, 1] => movie_id
        users = self.user_factors(x[:, 0])
        movies = self.movie_factors(x[:, 1])
        res = (users * movies).sum(dim=1, keepdim=True)
        res += self.user_bias(x[:, 0]) + self.movie_bias(x[:, 1])
        return sigmoid_range(res, *self.y_range)

In [94]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.953001,0.931553,00:07
1,0.838029,0.849181,00:07
2,0.594066,0.850282,00:07
3,0.414609,0.873276,00:07
4,0.287050,0.879537,00:07


In [122]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.955867,0.940355,00:06
1,0.868366,0.864493,00:07
2,0.745591,0.820899,00:06
3,0.595089,0.807670,00:06
4,0.494970,0.808947,00:07


In [96]:
class T(Module):
    def __init__(self):
        self.a = torch.ones(3)

In [99]:
L(T().parameters())

(#0) []

In [100]:
class T(Module):
    def __init__(self):
        self.a = nn.Parameter(torch.ones(3))

In [101]:
L(T().parameters())

(#1) [Parameter containing:
tensor([1., 1., 1.], requires_grad=True)]

In [102]:
class T(Module):
    def __init__(self):
        self.a = nn.Linear(1, 3, bias=False)

In [104]:
t = T()
L(t.parameters())

(#1) [Parameter containing:
tensor([[-0.8586],
        [ 0.8295],
        [-0.7262]], requires_grad=True)]

In [105]:
type(t.a.weight)

torch.nn.parameter.Parameter

In [106]:
def create_params(size):
    return nn.Parameter(torch.zeros(*size).normal_(0, 0.01))

In [110]:
create_params([3, 5]).shape

torch.Size([3, 5])

In [125]:
class DotProductBias(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0, 5.5)):
        self.user_factors = create_params([n_users, n_factors])
        self.user_bias = create_params([n_users])

        self.movie_factors = create_params([n_movies, n_factors])
        self.movie_bias = create_params([n_movies])

        self.y_range = y_range
    
    def forward(self, x):
        # x[batch, 0] => user_id
        # x[batch, 1] => movie_id
        users = self.user_factors[x[:, 0]]
        movies = self.movie_factors[x[:, 1]]
        res = (users * movies).sum(dim=1)
        res += self.user_bias[x[:, 0]] + self.movie_bias[x[:, 1]]                                                         
        return sigmoid_range(res, *self.y_range)

In [126]:
model = DotProductBias(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.948911,0.932214,00:07
1,0.857541,0.861588,00:07
2,0.737865,0.824385,00:07
3,0.568469,0.812904,00:07
4,0.456174,0.813951,00:07


In [127]:
movie_bias = learn.model.movie_bias.squeeze()
movie_bias.shape

torch.Size([1665])

In [130]:
idxs = movie_bias.argsort()[:5]
idxs

tensor([ 295,  850, 1001,  561,  140], device='cuda:0')

In [131]:
movie_bias[idxs]

tensor([-0.3588, -0.3142, -0.2913, -0.2910, -0.2799], device='cuda:0',
       grad_fn=<IndexBackward>)

In [133]:
[dls.classes['title'][i] for i in idxs]

['Children of the Corn: The Gathering (1996)',
 'Lawnmower Man 2: Beyond Cyberspace (1996)',
 'Mortal Kombat: Annihilation (1997)',
 'Free Willy 3: The Rescue (1997)',
 'Beautician and the Beast, The (1997)']

In [134]:
idxs = movie_bias.argsort(descending=True)[:5]
idxs

tensor([1399, 1501,   99, 1282, 1318], device='cuda:0')

In [135]:
movie_bias[idxs]

tensor([0.5826, 0.5511, 0.5257, 0.5125, 0.4976], device='cuda:0',
       grad_fn=<IndexBackward>)

In [136]:
[dls.classes['title'][i] for i in idxs]

['Star Wars (1977)',
 'Titanic (1997)',
 'As Good As It Gets (1997)',
 "Schindler's List (1993)",
 'Shawshank Redemption, The (1994)']

## Using fastai.collab